In [1]:
import pyspark
from pyspark.sql.functions import *
sc = pyspark.SparkContext('local[*]')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2024-11-26T02:04:45,215 WARN [Thread-4] org.apache.hadoop.util.NativeCodeLoader - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.sql.functions import *

In [3]:
# import findspark
# findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [4]:
# 1
cleaned_olist_customers_dataset = spark.read.csv("/obligatorio/data/cleaned/cleaned_olist_customers_dataset.csv", header=True, inferSchema=True)
# 2
cleaned_olist_geolocation_dataset = spark.read.csv("/obligatorio/data/cleaned/cleaned_olist_geolocation_dataset.csv", header=True, inferSchema=True)
# 3
cleaned_olist_order_items_dataset = spark.read.csv("/obligatorio/data/cleaned/cleaned_olist_order_items_dataset.csv", header=True, inferSchema=True)
# 4
cleaned_olist_order_payments_dataset = spark.read.csv("/obligatorio/data/cleaned/cleaned_olist_order_payments_dataset.csv", header=True, inferSchema=True)
# 5
cleaned_olist_order_reviews_dataset = spark.read.csv("/obligatorio/data/cleaned/cleaned_olist_order_reviews_dataset.csv", header=True, inferSchema=True)
# 6
cleaned_olist_orders_dataset = spark.read.csv("/obligatorio/data/cleaned/cleaned_olist_orders_dataset.csv", header=True, inferSchema=True)
# 7
cleaned_olist_products_dataset = spark.read.csv("/obligatorio/data/cleaned/cleaned_olist_products_dataset.csv", header=True, inferSchema=True)
# 8
cleaned_olist_sellers_dataset = spark.read.csv("/obligatorio/data/cleaned/olist_sellers_dataset.csv", header=True, inferSchema=True)
# 9
cleaned_product_category_name_translation_dataset = spark.read.csv("/obligatorio/data/cleaned/cleaned_product_category_name_translation.csv", header=True, inferSchema=True)

In [5]:
cleaned_olist_customers_dataset = spark.read.csv("/obligatorio/data/cleaned/cleaned_olist_customers_dataset.csv", header=True, inferSchema=True)

Es también conocido como diagrama estrella o diagrama de copo de nieve. Tenemos dos tipos de tablas, las de hechos, y las de dimensiones.

- Tablas de hechos: Contienen las medidas o los valores numéricos que se quieren analizar. Por lo general, los datos numéricos son agregados y analizados en diferentes niveles de granularidad, como por ejemplo, por día, semana, mes o trimestre. Estas tablas suelen tener muchas filas y relativamente pocas columnas, con cada fila representando una transacción u otra actividad registrada en el sistema.

- Tablas de dimensiones: Contienen información descriptiva que se utiliza para filtrar o segmentar los datos de la tabla de hechos. Estas tablas contienen atributos que describen las características de las transacciones, como la fecha, el lugar, el producto, el cliente, entre otros. Las tablas de dimensiones suelen tener menos filas y más columnas que las tablas de hechos.

Las tablas de hechos y las tablas de dimensiones están relacionadas entre sí a través de claves. Cada fila de la tabla de hechos se relaciona con una o más filas de la tabla de dimensiones a través de claves. Esta relación permite combinar los datos de las tablas de hechos y de dimensiones para realizar análisis detallados.

Damos forma a un esquema de estrella. Nuestras tablas de hechos van a ser el libro que es prestado y la ubicación del Libro. Nuestras dimensiones van a ser una de libros, una de librerías, otra de quienes piden prestados los libros y otra de publicadores.

Usamos claves surrogadas porque no tenemos control sobre los identificadores primarios de la fuente original. Así nos protegemos ante eventuales cambios.

Las tablas de dimensiones están hechas de tipo 1. Si queremos pasar a tipo 2 debemos agregar una columna flag que indique que estado de la fila es, o usar columnas con fechas para guardar el período de tiempo en el que estuvo el registro en ese estado, antes de pasar al siguiente.

Preguntas:

- ¿Conjunto de productos que más se venden juntos?
- Top 5 productos más vendidos por ciudad.
- ¿Qué días del mes se vende más?
- ¿Qué proporción de pedidos recibe cada calificación en las reseñas (de 1 a 5)? 
- ¿Cuál es el tiempo promedio entre la aprobación de un pedido y la entrega al cliente?

In [6]:
########################################################################################################################################

Database Schema

In [7]:
# Geolocalización

In [8]:
cleaned_olist_geolocation_dataset.printSchema()

root
 |-- geolocation_zip_code_prefix: integer (nullable = true)
 |-- geolocation_lat: double (nullable = true)
 |-- geolocation_lng: double (nullable = true)
 |-- geolocation_city: string (nullable = true)
 |-- geolocation_state: string (nullable = true)



In [9]:
dim_geolocation = cleaned_olist_geolocation_dataset.select(
    col("geolocation_zip_code_prefix").alias("zip_code_prefix"),
    col("geolocation_city").alias("city"),
    col("geolocation_state").alias("state"),
    col("geolocation_lat").alias("latitude"),
    col("geolocation_lng").alias("longitude")
).distinct().withColumn("geo_id", monotonically_increasing_id())

In [10]:
# Customer

In [11]:
cleaned_olist_customers_dataset.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: integer (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)



In [12]:
dim_customers = cleaned_olist_customers_dataset.join(
    dim_geolocation,
    cleaned_olist_customers_dataset.customer_zip_code_prefix == dim_geolocation.zip_code_prefix,
    "left"
).select(
    col("customer_id"),
    col("customer_unique_id"),
    col("geo_id").alias("customer_geo_id")
)

In [13]:
dim_customers.show(10)

+--------------------+--------------------+---------------+
|         customer_id|  customer_unique_id|customer_geo_id|
+--------------------+--------------------+---------------+
|369708cabd9831ea6...|94b731a41867b47c3...|          17071|
|369708cabd9831ea6...|94b731a41867b47c3...|          83611|
|369708cabd9831ea6...|94b731a41867b47c3...|         199901|
|369708cabd9831ea6...|94b731a41867b47c3...|         247945|
|369708cabd9831ea6...|94b731a41867b47c3...|         286101|
|369708cabd9831ea6...|94b731a41867b47c3...|         308830|
|369708cabd9831ea6...|94b731a41867b47c3...|         321192|
|369708cabd9831ea6...|94b731a41867b47c3...|         336608|
|369708cabd9831ea6...|94b731a41867b47c3...|         337947|
|369708cabd9831ea6...|94b731a41867b47c3...|     8589957421|
+--------------------+--------------------+---------------+
only showing top 10 rows



In [14]:
# Seller

In [15]:
cleaned_olist_sellers_dataset.printSchema()

root
 |-- seller_id: string (nullable = true)
 |-- seller_zip_code_prefix: integer (nullable = true)
 |-- seller_city: string (nullable = true)
 |-- seller_state: string (nullable = true)



In [16]:
dim_sellers = cleaned_olist_sellers_dataset.join(
    dim_geolocation,
    cleaned_olist_sellers_dataset.seller_zip_code_prefix == dim_geolocation.zip_code_prefix,
    "left"
).select(
    col("seller_id"),
    col("geo_id").alias("seller_geo_id")
)

In [17]:
dim_sellers.show(10)

+--------------------+-------------+
|           seller_id|seller_geo_id|
+--------------------+-------------+
|67883baaae6134ee8...|        17597|
|67883baaae6134ee8...|        35437|
|67883baaae6134ee8...|        48345|
|67883baaae6134ee8...|        53837|
|67883baaae6134ee8...|        55425|
|67883baaae6134ee8...|       105282|
|67883baaae6134ee8...|       120560|
|67883baaae6134ee8...|       157312|
|67883baaae6134ee8...|       160905|
|67883baaae6134ee8...|       170686|
+--------------------+-------------+
only showing top 10 rows



In [18]:
# Products

In [19]:
cleaned_olist_products_dataset.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: integer (nullable = true)
 |-- product_length_cm: integer (nullable = true)
 |-- product_height_cm: integer (nullable = true)
 |-- product_width_cm: integer (nullable = true)



In [20]:
cleaned_product_category_name_translation_dataset.printSchema()

root
 |-- product_category_name: string (nullable = true)
 |-- product_category_name_english: string (nullable = true)



In [21]:
dim_products = cleaned_olist_products_dataset.join(
    cleaned_product_category_name_translation_dataset,
    "product_category_name",
    "left"
).select(
    col("product_id"),
    col("product_category_name"),
    col("product_category_name_english"),
    col("product_name_lenght"),
    col("product_description_lenght"),
    col("product_photos_qty"),
    col("product_weight_g"),
    col("product_length_cm"),
    col("product_height_cm"),
    col("product_width_cm")
)

In [22]:
dim_sellers.show(10)

+--------------------+-------------+
|           seller_id|seller_geo_id|
+--------------------+-------------+
|67883baaae6134ee8...|        17597|
|67883baaae6134ee8...|        35437|
|67883baaae6134ee8...|        48345|
|67883baaae6134ee8...|        53837|
|67883baaae6134ee8...|        55425|
|67883baaae6134ee8...|       105282|
|67883baaae6134ee8...|       120560|
|67883baaae6134ee8...|       157312|
|67883baaae6134ee8...|       160905|
|67883baaae6134ee8...|       170686|
+--------------------+-------------+
only showing top 10 rows



In [23]:
# Payments

In [24]:
cleaned_olist_order_payments_dataset.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- payment_sequential: integer (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- payment_installments: integer (nullable = true)
 |-- payment_value: double (nullable = true)



In [25]:
cleaned_olist_order_payments_dataset.show(10)

+--------------------+------------------+------------+--------------------+-------------+
|            order_id|payment_sequential|payment_type|payment_installments|payment_value|
+--------------------+------------------+------------+--------------------+-------------+
|9389b64e7d922e3b0...|                 1| credit_card|                   4|       125.43|
|91e725244b1ed7224...|                 1| credit_card|                   3|       113.33|
|ad9447ca1bb7d23ed...|                 1|      boleto|                   1|       224.45|
|41b03a6bad9f1feda...|                 1| credit_card|                   1|       220.66|
|9378f7fb4dad9527f...|                 1| credit_card|                   1|       117.01|
|dd2abc99a0eb03ae7...|                 1| credit_card|                   3|        71.36|
|de806f0367f37cd3f...|                 1| credit_card|                   7|       157.13|
|4a49f0eb11c5d69be...|                 1| credit_card|                   1|        73.72|
|e56069963

In [26]:
dim_payments = cleaned_olist_order_payments_dataset.select("payment_type").distinct() \
    .withColumn("payment_type_id", monotonically_increasing_id())

In [27]:
dim_payments.show(10)

+------------+---------------+
|payment_type|payment_type_id|
+------------+---------------+
|      boleto|              0|
| not_defined|              1|
| credit_card|              2|
|     voucher|              3|
|  debit_card|              4|
+------------+---------------+



In [28]:
# Reviews

In [29]:
cleaned_olist_order_reviews_dataset.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- review_score: string (nullable = true)
 |-- review_comment_title: string (nullable = true)
 |-- review_comment_message: string (nullable = true)
 |-- review_creation_date: string (nullable = true)
 |-- review_answer_timestamp: string (nullable = true)



In [30]:
dim_reviews = cleaned_olist_order_reviews_dataset.select("*")

In [31]:
dim_reviews.show(5)

+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|           review_id|            order_id|review_score|review_comment_title|review_comment_message|review_creation_date|review_answer_timestamp|
+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|8f9aa50ee40780ba3...|729c445f5f350f88d...|           5|           Muito bom|  Creme maravilhoso...| 2018-05-10 00:00:00|    2018-05-10 16:21:17|
|38bcaf95057f842c3...|ca36cc19c0a413c8f...|           3|                null|  Só recebi um prod...| 2018-04-28 00:00:00|    2018-04-29 01:11:49|
|08ee947fb131a0a99...|0666806082c797ed8...|           5|                null|  Produto lindo, ótimo| 2017-07-08 00:00:00|    2017-07-09 09:59:54|
|1f0c5d31b49a18cc2...|f810273bc84ceae52...|           4|                null|  Entrega uma dia a...| 2018-03-11 00:00:00|   

In [32]:
cleaned_olist_order_reviews_dataset.show(5)

+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|           review_id|            order_id|review_score|review_comment_title|review_comment_message|review_creation_date|review_answer_timestamp|
+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|8f9aa50ee40780ba3...|729c445f5f350f88d...|           5|           Muito bom|  Creme maravilhoso...| 2018-05-10 00:00:00|    2018-05-10 16:21:17|
|38bcaf95057f842c3...|ca36cc19c0a413c8f...|           3|                null|  Só recebi um prod...| 2018-04-28 00:00:00|    2018-04-29 01:11:49|
|08ee947fb131a0a99...|0666806082c797ed8...|           5|                null|  Produto lindo, ótimo| 2017-07-08 00:00:00|    2017-07-09 09:59:54|
|1f0c5d31b49a18cc2...|f810273bc84ceae52...|           4|                null|  Entrega uma dia a...| 2018-03-11 00:00:00|   

In [33]:
# Order

In [34]:
dim_customers.show(4)

+--------------------+--------------------+---------------+
|         customer_id|  customer_unique_id|customer_geo_id|
+--------------------+--------------------+---------------+
|369708cabd9831ea6...|94b731a41867b47c3...|          17071|
|369708cabd9831ea6...|94b731a41867b47c3...|          83611|
|369708cabd9831ea6...|94b731a41867b47c3...|         199901|
|369708cabd9831ea6...|94b731a41867b47c3...|         247945|
+--------------------+--------------------+---------------+
only showing top 4 rows



In [41]:
# Unir las tablas necesarias
fact_orders = cleaned_olist_orders_dataset.join(cleaned_olist_order_items_dataset, "order_id") \
    .join(cleaned_olist_order_payments_dataset, "order_id", "left") \
    .join(cleaned_olist_order_reviews_dataset, "order_id", "left") \
    .join(dim_sellers, "seller_id", "left") \
    .join(dim_customers, "customer_id", "left") \
    .groupBy(
        "order_id", 
        "customer_id", 
        "seller_id",
        "review_id",
        "order_status",
        "customer_geo_id",
        "seller_geo_id",
        "product_id",
        "order_purchase_timestamp", 
        "order_approved_at", 
        "order_delivered_customer_date"
    ).agg(
        sum("price").alias("total_price"),
        sum("freight_value").alias("freight_value"),
        sum("payment_value").alias("total_payment_value"),  
        avg("review_score").alias("review_score")
    )

In [42]:
fact_orders.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- customer_geo_id: long (nullable = true)
 |-- seller_geo_id: long (nullable = true)
 |-- product_id: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- total_price: double (nullable = true)
 |-- freight_value: double (nullable = true)
 |-- total_payment_value: double (nullable = true)
 |-- review_score: double (nullable = true)



In [43]:
directory_results = "obligatorio/data/processed"

In [44]:
_path = f"hdfs://localhost:9000/{directory_results}/"
_path

'hdfs://localhost:9000/obligatorio/data/processed/'

In [46]:
cleaned_olist_customers_dataset.write.csv(_path+"/fact_orders.csv", header=True, mode="overwrite")

In [48]:
cleaned_olist_customers_dataset.count()

99441